This notebook is my practise to do EDA (exploratory data analysis), the first step in data science.

I used to be interested on daily covid data when covid hit Indonesia back in March and April. It had been a while, so let's have a look at them now

Data will be scrapped from:
* https://github.com/CSSEGISandData/COVID-19
* Our World in Data (for Indonesia): https://github.com/owid/covid-19-data/tree/master/public/data
* Covid data for Indonesia from SINTA (via KawalCOVID): http://sinta.ristekbrin.go.id/covid/datasets
* GEOjson for Indonesia: https://bitbucket.org/rifani/geojson-political-indonesia/src/master/

Summary of this notebook:
* Download Covid19 data for worldwide and Indonesia from several sources
* Download GEOjson map for Indonesia
* Worldwide: summary of latest data, worldwide map, death vs confirmed comparison
* Indonesia: summary of latest data, summary plots (total cases, daily cases, positive rate and mortality rate) and other random stats that I am interested in

New skills I picked up and applied on this notebook:
* First time using Git properly
* Using Plotly Express
* Extracting data from Google Sheet API
* Cleaning data. The spreadsheet is messy. Table are stacked on other tables in the same spreadsheet tab
* Extracted data is string. Not sure if there is a way to extract in a numeric format instead of converting it to float manually. For next time, maybe there is a way to just download from Google Sheet automatically and just pd.read_csv()
* Working with GEOjson data format and plotting an interactive map


# Covid19 in Indonesia

# Import necessary python libraries

In [18]:
#collapse
# download python libraries
from datetime import datetime, timedelta
import os
import glob
import wget
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import json
import plotly.express as px
import plotly.graph_objs as go

# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

from IPython.display import HTML

# Import data

In [19]:
#collapse
# Download data from Github (daily)
os.chdir("C:/Users/Riyan Aditya/Desktop/ML_learning/Project4_EDA_Covid_Indo/datasets")

os.remove('time_series_covid19_confirmed_global.csv')
os.remove('time_series_covid19_deaths_global.csv')
os.remove('time_series_covid19_recovered_global.csv')

# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

# download files
for url in urls:
    filename = wget.download(url)

100% [............................................................................] 265849 / 265849

# Clean & preprocess data

In [20]:
#collapse
# convert csv to df
confirmed_global = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_global = pd.read_csv('time_series_covid19_deaths_global.csv')
recovered_global = pd.read_csv('time_series_covid19_recovered_global.csv')

In [21]:
#collapse
# Melt DF => switch rows of dates into column for simpler DF
dates = confirmed_global.columns[4:]

confirmed_globalv2 = confirmed_global.melt(id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'],
                                          value_vars = dates, var_name ='Date', value_name = 'Confirmed')
deaths_globalv2 = deaths_global.melt(id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'],
                                          value_vars = dates, var_name ='Date', value_name = 'Deaths')
recovered_globalv2 = recovered_global.melt(id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'],
                                          value_vars = dates, var_name ='Date', value_name = 'Recovered')
print(confirmed_globalv2.shape)
print(deaths_globalv2.shape)
print(recovered_globalv2.shape)

(70755, 6)
(70755, 6)
(67310, 6)


Why are there differences in number of rows between confirmed (or death) & recovered?

This seems to suggest some countries are missing their data

In [22]:
#collapse
# Combine df
covid_global = confirmed_globalv2.merge(deaths_globalv2, how='left', on = 
                                        ['Province/State', 'Country/Region', 'Lat', 'Long','Date']).merge(
                                        recovered_globalv2, how='left', on =
                                        ['Province/State', 'Country/Region', 'Lat', 'Long','Date'])

In [23]:
#collapse
# preprocessing
covid_global['Date'] = pd.to_datetime(covid_global['Date'])

#active cases
covid_global['Active'] = covid_global['Confirmed'] - covid_global['Deaths'] - covid_global['Recovered']

In [24]:
#collapse
# Data by day
covid_global_daily = covid_global.groupby('Date')['Confirmed','Deaths','Recovered','Active'].sum().reset_index()

In [25]:
#collapse
# Data by country
temp = covid_global[covid_global['Date'] ==max(covid_global['Date'])].reset_index(drop=True).drop('Date', axis = 1)
covid_global_percountry = temp.groupby('Country/Region')['Confirmed','Deaths','Recovered','Active'].sum().reset_index()

# Worldwide Data Viz

## Latest data

In [26]:
#collapse
# latest data
print('Date today',covid_global_daily['Date'].iloc[-1])
print('Total cases','{:,}'.format(covid_global_daily['Confirmed'].iloc[-1]))
print('Active cases','{:,}'.format(covid_global_daily['Active'].iloc[-1]))
print('Recovered cases','{:,}'.format(covid_global_daily['Recovered'].iloc[-1]))
print('Deaths cases','{:,}'.format(covid_global_daily['Deaths'].iloc[-1]))

Date today 2020-10-12 00:00:00
Total cases 37,801,526
Active cases 9,224,575.0
Recovered cases 26,108,249.0
Deaths cases 997,143.0


In [27]:
#collapse
# plot
temp = covid_global_daily[['Date','Deaths','Recovered','Active']].tail(1)
temp = temp.melt(id_vars='Date',value_vars = ['Active','Deaths','Recovered'])
fig = px.treemap(temp, path=['variable'],values = 'value', height = 225)
fig.data[0].textinfo = 'label+text+value'

In [28]:
#collapse
HTML(fig.to_html(include_plotlyjs='cdn'))

## Total confirmed cases world map

In [38]:
#collapse
def plot_map(df, col, pal):
    df = df[df[col]>0]
    fig2 = px.choropleth(df, locations="Country/Region", locationmode='country names', 
                  color=col, hover_name="Country/Region", 
                  title=col, hover_data=[col], color_continuous_scale=pal)
    fig2.update_layout(coloraxis_showscale=False)
    return fig2
    

In [39]:
#collapse
fig2 = plot_map(covid_global_percountry, 'Confirmed', 'matter')
HTML(fig2.to_html(include_plotlyjs='cdn'))

## Treemap total confirmed cases 

In [ ]:
#collapse
def plot_treemap(df,col):
    fig = px.treemap(df, path=["Country/Region"], values=col, height=700,
                 title=col, color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.data[0].textinfo = 'label+text+value'
    

In [ ]:
#collapse
plot_treemap(covid_global_percountry,'Confirmed')
HTML(fig.to_html(include_plotlyjs='cdn'))

## Death vs confirmed

For top 50 countries with the highest total cases

In [ ]:
#collapse
def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.2f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

In [ ]:
#collapse
# plot
fig = px.scatter(covid_global_percountry.sort_values('Deaths', ascending=False).iloc[:50, :], 
                 x='Confirmed', y='Deaths', color='Country/Region', size='Confirmed', 
                 height=700, text='Country/Region', log_x=True, log_y=True, 
                 title='Deaths vs Confirmed (Scale is in log10)',
                hover_data={'Country/Region':True,'Confirmed':':,','Deaths':':,'})
fig.update_traces(textposition='top center')
fig.update_layout(showlegend=False)
HTML(fig.to_html(include_plotlyjs='cdn'))

# Indonesia

Data starts 18 Mar